In [7]:
import numpy as np
import pandas as pd
from collections import Counter

This dataset came from here: https://www.kaggle.com/datasets/pashupatigupta/emotion-detection-from-text

In [8]:
df = pd.read_csv("tweet_emotions.csv")
df.head()

,tweet_id,sentiment,content
0,1956967341,empty,@tiffanylue i know i was listenin to bad habi...
1,1956967666,sadness,Layin n bed with a headache ughhhh...waitin o...
2,1956967696,sadness,Funeral ceremony...gloomy friday...
3,1956967789,enthusiasm,wants to hang out with friends SOON!
4,1956968416,neutral,@dannycastillo We want to trade with someone w...


In [9]:
Counter(df['sentiment'])

Counter({'empty': 827,
         'sadness': 5165,
         'enthusiasm': 759,
         'neutral': 8638,
         'worry': 8459,
         'surprise': 2187,
         'love': 3842,
         'fun': 1776,
         'hate': 1323,
         'happiness': 5209,
         'boredom': 179,
         'relief': 1526,
         'anger': 110})

In [23]:
df[df['sentiment'] == 'sadness']['content']

1        Layin n bed with a headache  ughhhh...waitin o...
2                      Funeral ceremony...gloomy friday...
6        I should be sleep, but im not! thinking about ...
8                  @charviray Charlene my love. I miss you
9               @kelcouch I'm sorry  at least it's Friday?
                               ...                        
39882        Heaps keen for next weekend.     Mummy. &lt;3
39894    @renagades Have a great day..... if the rain k...
39909    @Ssisch Hmmm... Sisch &amp; sound design. So w...
39939    @watermelon39 haha! And Twitter! Hard though i...
39965                           is heading off to the fair
Name: content, Length: 5165, dtype: object

In [27]:
df[df['sentiment'] == 'sadness']['content'][1]

'Layin n bed with a headache  ughhhh...waitin on your call...'